# 1. 중간고사 범위에 수집된 데이터 + 날씨 데이터
- 중간고사 때 수행했던 창원시 날씨 데이터 외에 경상도,충청도 지역 확대 및 날씨 데이터 수집/JOIN


In [4]:
import pandas as pd
import os

# df를 처리된 CSV 파일에서 다시 초기화하고 '판매일'을 datetime으로 변환
df = pd.read_csv("./data/df_processed_merged.csv")
df['판매일'] = pd.to_datetime(df['판매일'])

# base_weather_path 재정의
base_weather_path = './data/weather'

# load_and_process_weather_data 함수가 스코프 내에 있도록 재정의
def load_and_process_weather_data(directory_path):
    """
    디렉토리에서 모든 CSV 파일을 읽고, 연결하며, 날짜 열의 이름을 '판매일'로 변경하고,
    '판매일'을 datetime 형식으로 변환하며, 추가적인 기상 열을 추출하고 표준화합니다.
    """
    all_files = os.listdir(directory_path)
    df_list = []

    for filename in all_files:
        if filename.endswith('.csv'):
            file_path = os.path.join(directory_path, filename)
            try:
                # 'cp949' 인코딩으로 읽기 시도
                df_temp = pd.read_csv(file_path, encoding='cp949')
                df_list.append(df_temp)
            except UnicodeDecodeError:
                try:
                    # 'cp949' 실패 시 'euc-kr'로 대체
                    df_temp = pd.read_csv(file_path, encoding='euc-kr')
                    df_list.append(df_temp)
                except Exception as e:
                    print(f"cp949와 euc-kr 모두 {file_path} 읽기 오류: {e}")
            except Exception as e:
                print(f"{file_path} 읽기 오류: {e}")

    if not df_list:
        print(f"{directory_path} 에서 CSV 파일을 찾거나 처리하지 못했습니다.")
        return pd.DataFrame()

    combined_df = pd.concat(df_list, ignore_index=True)

    # 날짜 열 이름을 '판매일'로 변경
    if '일시' in combined_df.columns:
        combined_df = combined_df.rename(columns={'일시': '판매일'})
    elif '날짜' in combined_df.columns:
        combined_df = combined_df.rename(columns={'날짜': '판매일'})

    # '판매일'을 datetime 형식으로 변환
    if '판매일' in combined_df.columns:
        combined_df['판매일'] = pd.to_datetime(combined_df['판매일'], errors='coerce')

    # '평균기온(°C)' 열 이름을 '평균기온'으로 변경
    if '평균기온(°C)' in combined_df.columns:
        combined_df = combined_df.rename(columns={'평균기온(°C)': '평균기온'})

    # 유지하고 표준화할 대상 열 정의
    target_columns = [
        '판매일',
        '평균기온',
        '평균 상대습도(%)',
        '일강수량(mm)',
        '평균 풍속(m/s)',
        '합계 일조시간(hr)',
        '평균 지면온도(°C)'
    ]

    # combined_df에 존재하는 대상 열만 필터링
    existing_target_columns = [col for col in target_columns if col in combined_df.columns]
    if '판매일' not in existing_target_columns and '판매일' in combined_df.columns:
        existing_target_columns.insert(0, '판매일')

    processed_df = combined_df[existing_target_columns].copy()

    # 지정된 열들을 숫자 형식으로 변환 (오류 발생 시 NaN 처리)
    numeric_cols = [
        '평균기온',
        '평균 상대습도(%)',
        '일강수량(mm)',
        '평균 풍속(m/s)',
        '합계 일조시간(hr)',
        '평균 지면온도(°C)'
    ]

    for col in numeric_cols:
        if col in processed_df.columns:
            processed_df[col] = pd.to_numeric(processed_df[col], errors='coerce')

    return processed_df

sigungu_weather_dfs = []
# df에서 고유한 '시군' 값 가져오기
unique_sigungu = df['시군'].unique()

print("재정의된 함수로 시군별 날씨 데이터를 처리 중...")
for sigungu in unique_sigungu:
    sigungu_path = os.path.join(base_weather_path, sigungu)
    if os.path.isdir(sigungu_path):
        # 시군별 날씨 데이터 로드 및 처리
        weather_df = load_and_process_weather_data(sigungu_path)
        if not weather_df.empty:
            weather_df['시군'] = sigungu
            sigungu_weather_dfs.append(weather_df)
            print(f"  {sigungu} 날씨 데이터 처리됨")
    # else: # 콘솔 출력을 줄이기 위해 주석 처리
    #     print(f"  {sigungu} 에 대한 날씨 데이터 디렉토리가 없습니다")

print("시군별 날씨 데이터 수집 완료.")

if sigungu_weather_dfs:
    # 모든 시군별 날씨 데이터 연결
    all_sigungu_weather_data = pd.concat(sigungu_weather_dfs, ignore_index=True)
    # '판매일'과 '시군' 기준으로 중복 제거하여 중복 병합 방지
    all_sigungu_weather_data.drop_duplicates(subset=['판매일', '시군'], inplace=True)
    print("시군별 날씨 데이터 병합됨.")
else:
    # 데이터가 로드되지 않은 경우, 예상되는 모든 열을 가진 빈 DataFrame 정의
    all_sigungu_weather_data = pd.DataFrame(columns=[
        '판매일', '시군', '평균기온', '평균 상대습도(%)', '일강수량(mm)',
        '평균 풍속(m/s)', '합계 일조시간(hr)', '평균 지면온도(°C)'
    ])
    print("시군별 날씨 데이터가 성공적으로 로드되지 않았습니다.")

# 병합할 새로운 날씨 열 목록 정의 ('판매일', '시군' 제외)
weather_cols_to_merge = [
    '평균기온', '평균 상대습도(%)', '일강수량(mm)',
    '평균 풍속(m/s)', '합계 일조시간(hr)', '평균 지면온도(°C)'
]

# 병합을 위한 열 준비: '판매일', '시군' + 모든 weather_cols_to_merge
merge_columns = ['판매일', '시군'] + weather_cols_to_merge

# df에 날씨 데이터를 가져오기 위해 left merge 수행
df_merged_sigungu = pd.merge(df, all_sigungu_weather_data[merge_columns],
                             on=['판매일', '시군'],
                             how='left', suffixes=('_original', '_sigungu'))

# 병합된 '평균기온'을 사용하여 df의 '기온' 열 업데이트
df['기온'] = df_merged_sigungu['평균기온'].fillna(df['기온'])

# df에 새로운 날씨 열을 업데이트하거나 추가하고, 기존 값이 있으면 NaN 채우기
for col in weather_cols_to_merge:
    # 병합된 DataFrame에 열이 있는 경우에만 업데이트/추가
    if col in df_merged_sigungu.columns:
        # 업데이트를 위해 원래 열 이름 사용
        df[col] = df_merged_sigungu[col].fillna(df.get(col, pd.NA))

print("시군별 날씨 데이터를 사용하여 '기온'을 업데이트하고 새로운 날씨 열을 추가했습니다.")

# 업데이트 확인
display(df.head())
print(df.info())

재정의된 함수로 시군별 날씨 데이터를 처리 중...
  창원시 날씨 데이터 처리됨
  포항시 날씨 데이터 처리됨
  제주시 날씨 데이터 처리됨
  김해시 날씨 데이터 처리됨
  창녕군 날씨 데이터 처리됨
  함안군 날씨 데이터 처리됨
  고성군 날씨 데이터 처리됨
  의령군 날씨 데이터 처리됨
  통영시 날씨 데이터 처리됨
  진주시 날씨 데이터 처리됨
  밀양시 날씨 데이터 처리됨
  합천군 날씨 데이터 처리됨
  평택시 날씨 데이터 처리됨
  거제시 날씨 데이터 처리됨
  남해군 날씨 데이터 처리됨
  사천시 날씨 데이터 처리됨
  음성군 날씨 데이터 처리됨
  김포시 날씨 데이터 처리됨
  경주시 날씨 데이터 처리됨
  구미시 날씨 데이터 처리됨
  경산시 날씨 데이터 처리됨
  이천시 날씨 데이터 처리됨
  천안시 날씨 데이터 처리됨
  산청군 날씨 데이터 처리됨
시군별 날씨 데이터 수집 완료.
시군별 날씨 데이터 병합됨.
시군별 날씨 데이터를 사용하여 '기온'을 업데이트하고 새로운 날씨 열을 추가했습니다.


,판매일,EA,주소,시도,시군,구,기온,상품명_cleaned,제로구분,년월,평균기온,평균 상대습도(%),일강수량(mm),평균 풍속(m/s),합계 일조시간(hr),평균 지면온도(°C)
0,2021-01-31,30,경상남도 창원시 마산합포구 3·15대로,경상남도,창원시,마산합포구,5.4,칠성사이다,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0
1,2021-01-31,24,경상남도 창원시 마산합포구 3·15대로,경상남도,창원시,마산합포구,5.4,코카콜라,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0
2,2021-01-31,24,경상남도 창원시 마산합포구 3·15대로,경상남도,창원시,마산합포구,5.4,코카콜라,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0
3,2021-01-31,6,경상남도 창원시 의창구 우곡로,경상남도,창원시,의창구,5.4,펩시콜라,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0
4,2021-01-31,12,경상남도 창원시 마산회원구 양덕동12길,경상남도,창원시,마산회원구,5.4,스프라이트+코카콜라,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42871 entries, 0 to 42870
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   판매일          42871 non-null  datetime64[ns]
 1   EA           42871 non-null  int64         
 2   주소           42871 non-null  object        
 3   시도           42871 non-null  object        
 4   시군           42871 non-null  object        
 5   구            42871 non-null  object        
 6   기온           40350 non-null  float64       
 7   상품명_cleaned  42871 non-null  object        
 8   제로구분         42871 non-null  object        
 9   년월           42871 non-null  object        
 10  평균기온         40350 non-null  float64       
 11  평균 상대습도(%)   40347 non-null  float64       
 12  일강수량(mm)     16147 non-null  float64       
 13  평균 풍속(m/s)   40341 non-null  float64       
 14  합계 일조시간(hr)  40267 non-null  float64       
 15  평균 지면온도(°C)  40334 non-null  float64       
dtypes: d

In [5]:
sido_weather_dfs = []
# df에서 고유한 '시도' 값 가져오기
unique_sido = df['시도'].unique()

print("시도별 날씨 데이터 처리 중...")
for sido in unique_sido:
    sido_path = os.path.join(base_weather_path, sido)
    if os.path.isdir(sido_path):
        # 시도별 날씨 데이터 로드 및 처리
        weather_df = load_and_process_weather_data(sido_path)
        if not weather_df.empty:
            weather_df['시도'] = sido
            sido_weather_dfs.append(weather_df)
            print(f"  {sido} 날씨 데이터 처리됨")
    # else: # 콘솔 출력을 줄이기 위해 주석 처리
    #     print(f"  {sido} 에 대한 날씨 데이터 디렉토리가 없습니다")

print("시도별 날씨 데이터 수집 완료.")

if sido_weather_dfs:
    # 모든 시도별 날씨 데이터 연결
    all_sido_weather_data = pd.concat(sido_weather_dfs, ignore_index=True)
    # '판매일'과 '시도' 기준으로 중복 제거
    all_sido_weather_data.drop_duplicates(subset=['판매일', '시도'], inplace=True)
    print("시도별 날씨 데이터 병합됨.")
else:
    # 데이터가 로드되지 않은 경우, 예상되는 모든 열을 가진 빈 DataFrame 정의
    all_sido_weather_data = pd.DataFrame(columns=[
        '판매일', '시도', '평균기온', '평균 상대습도(%)', '일강수량(mm)',
        '평균 풍속(m/s)', '합계 일조시간(hr)', '평균 지면온도(°C)'
    ])
    print("시도별 날씨 데이터가 성공적으로 로드되지 않았습니다.")

# 병합/업데이트할 날씨 열 목록 정의
weather_cols_to_update = [
    '평균기온', '평균 상대습도(%)', '일강수량(mm)',
    '평균 풍속(m/s)', '합계 일조시간(hr)', '평균 지면온도(°C)'
]

# 병합을 위한 열 준비: '판매일', '시도' + 모든 weather_cols_to_update
merge_columns_sido = ['판매일', '시도'] + weather_cols_to_update

# df에 시도별 날씨 데이터를 가져오기 위해 left merge 수행
df_merged_sido = pd.merge(df, all_sido_weather_data[merge_columns_sido],
                            on=['판매일', '시도'],
                            how='left', suffixes=('_original', '_sido'))

# 병합된 DataFrame의 '평균기온_sido'를 사용하여 df의 '기온' 열에 있는 NaN 값 업데이트
if '평균기온_sido' in df_merged_sido.columns:
    df['기온'] = df['기온'].fillna(df_merged_sido['평균기온_sido'])

# 시도별 데이터로 df의 다른 날씨 열에 있는 NaN 값 채우기
for col in weather_cols_to_update:
    sido_merged_col_name = col + '_sido'

    if sido_merged_col_name in df_merged_sido.columns:
        # 해당 열이 df에 없으면 NaN으로 처리하여 fillna를 통해 열과 값을 효과적으로 추가
        df[col] = df[col].fillna(df_merged_sido[sido_merged_col_name])

print("폴백으로 시도별 날씨 데이터를 사용하여 '기온' 및 기타 날씨 열을 업데이트했습니다.")

# 업데이트 확인
display(df.head())
print(df.info())

시도별 날씨 데이터 처리 중...
  대구광역시 날씨 데이터 처리됨
  부산광역시 날씨 데이터 처리됨
  서울특별시 날씨 데이터 처리됨
  대전광역시 날씨 데이터 처리됨
  광주광역시 날씨 데이터 처리됨
  울산광역시 날씨 데이터 처리됨
시도별 날씨 데이터 수집 완료.
시도별 날씨 데이터 병합됨.
폴백으로 시도별 날씨 데이터를 사용하여 '기온' 및 기타 날씨 열을 업데이트했습니다.


,판매일,EA,주소,시도,시군,구,기온,상품명_cleaned,제로구분,년월,평균기온,평균 상대습도(%),일강수량(mm),평균 풍속(m/s),합계 일조시간(hr),평균 지면온도(°C)
0,2021-01-31,30,경상남도 창원시 마산합포구 3·15대로,경상남도,창원시,마산합포구,5.4,칠성사이다,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0
1,2021-01-31,24,경상남도 창원시 마산합포구 3·15대로,경상남도,창원시,마산합포구,5.4,코카콜라,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0
2,2021-01-31,24,경상남도 창원시 마산합포구 3·15대로,경상남도,창원시,마산합포구,5.4,코카콜라,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0
3,2021-01-31,6,경상남도 창원시 의창구 우곡로,경상남도,창원시,의창구,5.4,펩시콜라,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0
4,2021-01-31,12,경상남도 창원시 마산회원구 양덕동12길,경상남도,창원시,마산회원구,5.4,스프라이트+코카콜라,일반,2021-01,5.4,53.6,0.0,1.3,8.5,3.0


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 42871 entries, 0 to 42870
Data columns (total 16 columns):
 #   Column       Non-Null Count  Dtype         
---  ------       --------------  -----         
 0   판매일          42871 non-null  datetime64[ns]
 1   EA           42871 non-null  int64         
 2   주소           42871 non-null  object        
 3   시도           42871 non-null  object        
 4   시군           42871 non-null  object        
 5   구            42871 non-null  object        
 6   기온           42866 non-null  float64       
 7   상품명_cleaned  42871 non-null  object        
 8   제로구분         42871 non-null  object        
 9   년월           42871 non-null  object        
 10  평균기온         42866 non-null  float64       
 11  평균 상대습도(%)   42863 non-null  float64       
 12  일강수량(mm)     17132 non-null  float64       
 13  평균 풍속(m/s)   42853 non-null  float64       
 14  합계 일조시간(hr)  42779 non-null  float64       
 15  평균 지면온도(°C)  42850 non-null  float64       
dtypes: d

In [6]:
# 처리된 DataFrame을 CSV 파일로 저장할 경로 정의
output_path = "./outputs/data/df_1.csv"
# DataFrame을 CSV 파일로 저장 (인덱스 제외)
df.to_csv(output_path, index=False)
print(f"DataFrame을 {output_path} 에 성공적으로 저장했습니다.")

DataFrame을 ./outputs/data/df_1.csv 에 성공적으로 저장했습니다.
